In [ ]:
# Session identifier, do not remove or modify.
from yields.session import Session
session = Session("2cfe6e21-37c8-4794-8f44-a3702624db8b")

# EXPLAINABLE AI - understanding *why* a model makes certain predictions

**GENERAL** <br>

The assessment of *any* model falls into two broad categories:

* **How good does the model perform?** <br>
Here, we can rely on the typical metrics that assess the model quality such as 'accuracy', 'false positive ratio', 'precision' etc. Those metrics only look at the numbers coming out of the algorithm (whether it's a simple linear regression or a complex convolutional neural network) without bothering how the algorithm comes to these predictions (as long as it's accurate enough for the problem at hand, we're happy). Those analyses often maken use of the *combination of both the model predictions and the target* the model is trying to predict (in case of supervised learning problems). <br>
For this part we already have some notebooks available.

* **Why does the model make certain predictions?** <br>
Here, we try to understand the behaviour of the model *only on the predictions*. We do not care about the target, but we are *trying to understand how the different input variables impact the overall model prediction*. The outcomes of those analyses enable users to 'whiten the black-box' and get comfortable that the model at hand spits out plausible results, in the sense that those analyses can be benchmarked against human intuition. <br>
Note that the explainibility could (should) be integral part of any descent model assessment exercise. However, for the sake of clarity, we treat it separately inside this notebook.



**PURPOSE** <br>

This notebook offers some of the state-of-the-art 'explainable AI' techniques that **helps users understand how different inputs affect the model predictions.** <br>

This work relies on the work done on https://christophm.github.io/interpretable-ml-book/ .<br>

**INPUTS** <br>

The techniques demonstrated need the following ingredients:

* a trained model that can be read (e.g. as an artifact) into this notebook and it should have some 'predict' function (we will use a scikit-learn pipeline model that has the 'predict_proba' method to compute the predicted probabilities (for other model output types (e.g. regression) or environment used to train your model (e.g. Keras) - see other notebooks).
* a dataset on which the trained model can make predictions. We assume this dataset is nicely cleansed such that it does not contain any missing values or alike. This helps the plotting part.

As a running example, we use our own trained logistic regression model on the Titanic dataset, i.e. we are trying to predict the probability of survival given some passenger features.<br>

**USE CASE** <br>

The generic use-case is the following: given a trained model and a dataset on which this model is (or will be) applied, how can we ensure that the model behaves according to expectations/intuition? This question is relevant both during model development and assessment once in production. The techniques demonstrated below will help to identify potential bias in the prediction and identify potential counterintuitive behaviour.


# OVERVIEW OF THE DIFFERENT TECHNIQUES

**DEFINITIONS**

* Model agnostic: does not need to know about any underlying techniqes/algorithms implemented. Only needs model inputs and predictions and can thus be applied on any trained machine learning model.
* Global: gives a comprehensive explanation on the entire data set, describing the impact of feature(s) on the target variable in the context of the overall data.
* Local: describes the impact of feature(s) on an observation (i.e. the individual) level.

**OVERVIEW**

| XAI technique's name | Global | Local | Model agnostic | In a nutshell | Assumptions | Limitations |
| :- | :-: | :-: | :-: | :- | :- | :- |
| Model reliance (feature permutation) | Y | N | Y | For a given feature, randomly shuffle the values in the dataset over the observations. Measure the new model performance. Do this several times and plot the distribution of 'performance after shuffling'. Features that worsen the performance most are the 'most important variables'. | Independent features | The target is needed, so this one cannot be used for unsupervised models. |
| Partial Dependency Plots (DPD) | Y | N | Y | Replace the feature of interest with a fixed value over all observations in the dataset. Then average out the predictions. | Features (model inputs) are assumed to be independent. | * may not be accurate for values that are in intervals with very little data. <br> * Heterogenous (e.g. non-monotonically) effects between the features and the target will not be captured (see ICE for this).|
|Individual Conditional Expectation | N | Y | Y | How does the predicted outcome for a given observation change as we vary the feature value? Averaging of ICE over all observations yields the PDP | Independence of features | See PDP |
| Shapley values | Y | Y | Y | Explain which features make an individual prediction deviate from the average prediction over the dataset. Estimate the impact of a given model variable by averaging the prediction over all possible subsets of feature combinations ('coalitions') | * independent variables (no correlation) <br> | * Since the number of possible feature combinations is $2^N$ this might be very time consuming. <br> Does not work well when having lot's of one-hot encoded variables (e.g. sum of one-hot-encoded Shapley values for a single feature does not equal the Shapley value in the original representation. |


<a id="toc"></a>

## Table of Contents
1. [Setup](#set)     

    1.1 [Import Libraries](#imp_lib)  
        
    1.2 [Load input](#load_input)   
    
    1.3 [General model performance](#makepred)
    
2. [Feature permutation](#fp)
    
3. [Partial Dependency Plots](#pdp)
                
4. [Shapley values](#shapley)

5. [Diverse Counterfactual Explanations (DICE)](#dice)


<a id="set"></a>
# [Setup](#toc)

<a id="imp_lib"></a>
## [Import libraries](#toc)

In [ ]:
from yields.providers.filesystem import ProviderFactoryFilesystem

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
import pickle

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sklearn, json

from sklearn.inspection import permutation_importance

import shap

import plotly.express as px

<a id="load_input"></a>
## [Load inputs](#toc)

In [ ]:
#Aliases of inputs, outputs, parameters
session.io.inputs, session.io.outputs, session.io.parameters

In [ ]:
input_artifact_ds = session.io.input('ds')
file_handler = input_artifact_ds.file(input_artifact_ds.files[0]).open()
df = pd.read_csv(file_handler)

In [ ]:
input_artifact_dmm_ds = session.io.input('dmm_ds')
file_handler = input_artifact_dmm_ds.file(input_artifact_dmm_ds.files[0]).open()
schema = json.load(file_handler)

In [ ]:
target, continuous_inputs, categorical_inputs = schema['target'], schema['continuous_inputs'] , schema['categorical_inputs']
inputs = continuous_inputs + categorical_inputs
print('TARGET VARIABLE: ', target)
print('CONTINUOUS inputs: ', continuous_inputs)
print('COUNTABLE inputs: ', categorical_inputs)

In [ ]:
"""Loading the model that has been developed (by other) and for which we want to assess the explainability

FOR THE GIVEN NOTEBOOK TO WORK PROPERLY: it needs a 'predict_proba' method, say basically a sklearn trained model for binary target.
"""
input_artifact_dmm_ds = session.io.input('trained_model')
file_handler = input_artifact_dmm_ds.file(input_artifact_dmm_ds.files[0]).open()
model_to_explain = pickle.load(file_handler)

In [ ]:
""" PARAMETERS """

feature_importance_metric = session.io.parameter('metric to assess feature importance').value #'roc_auc'
preprocessor = session.io.parameter('name of preprocessor object in the model pipeline').value #'predict_proba'or
classifier = session.io.parameter('name of the model itself in the model pipeline').value

In [ ]:
input_artifact_preprocessor = session.io.input('preprocessor')
file_handler = input_artifact_preprocessor.file(input_artifact_preprocessor.files[0]).open()
trained_preprocessor = pickle.load(file_handler)

<a id="makepred"></a>
## [General model performance](#toc)

A descent model performance is always assumed before diving into explainability methods.<br>
It's off course non-sensical to "explain a model that's no better than a random guess". <br>

So let us first check some performance measure for the model at hand, for which we take the Area-under-the-ROC curve (binary outcome!). The higher the value, the better the model is able to discriminate between survivors and non-survivors. Values > 85% are typically considered very good. <br>

In [ ]:
#1. make predictions ('model_to_explain' is an sklearn object that has the method 'predict_proba' with computes the predicted probabilities for the binary model at hand).
predictions = model_to_explain.predict_proba(df[inputs])[:,1] #gives the prediction for the 'positive class'.
print("AUC of given model:" + str(sklearn.metrics.roc_auc_score(df[target] , predictions)))

<a id="fp"></a>
# [Feature permutation](#toc)

When, for a given feature, the values are shuffled over the observations, we might expect a worse model performance as compared to our trained model (by subtracting the new model performance (e.g. AUC, R², ...) from the one that we obtained during training).<br>
We can measure this worsening by repeatedly shuffling the values for a given feature and plot the distribution. This can off course be done for all features. Features that tend to increase the model performance deterioration most are deemed the most import features in predicting the outcome.<br>

Note that this method is model agnostic as it does not need the model specifics to do this shuffling. However, the outcome of this analyses can be benchmarked against the model specific results. For example, in linear regression, the coefficients (or weights) determine the feature importance. So the feature with the largest value is also expected to have the largest feature importance. However, when such an easy interpretation cannot be performed (e.g. deep feedforward networks), one might use the proposed feature importance method.

In [ ]:
r = permutation_importance(model_to_explain, df[inputs], df[target],  scoring = feature_importance_metric, n_repeats=30, random_state=0)
r_df = pd.DataFrame(data = r['importances'], index = inputs).T

fig = px.box(r_df, title = "Feature importance")
fig.update_xaxes(title= 'Model features')
fig.update_yaxes(title = "Model Performance (trained) - Model performance (shuffled)")
fig.show()

<a id="pdp"></a>
# [Partial Dependency & Individual Conditional Expectation plots](#toc)

In a partial dependency plot, we analyse the impact of every model input (feature) on the prediction by averaging out the model predictions over all possibilities of all the other features. Statistically speaking, we thus 'marginalize' over all the other (N-1) model inputs (assuming the model has N inputs).<br>

In **layman terms**, what happens is the following:

1. For every observation, replace the feature that is being investigated with the same value over all observations (e.g. replace 'age' for all observations with value 55). Note that all other variables are left untouched (note that there might be correlation between different features that we ignore by simply replacing age with a given value. This is an assumption that is made when creating PDP's).
2. Calculate the model predictions for every observation using the above replacement.
3. Take the average prediction outcome over all observations. This is the 'partial dependency corresponding to the value 'age = 55'.
4. Repeat steps 1-3 by looping over the range of interest for the given feature (say age can run from 0 - 80). When calculating the partial dependence for the range of values we're interested in, we finally get our partial dependency plot.

When we pick out a single observation and we follow it's prediction by changing the feature of interest, we have the so-called 'Individual Conditional Expectation (ICE) plot'. <br> 
Indeed, by changing a single value of a given model variable, we answer the question: 'given variable x has value y, how does this affect the prediction for the current observation'.<br>

**Result** <br>
Plot showing the qualitative and quantitative behaviour on the predicted outcome by changing the feature values over a given range. 

**We demonstrate the PDP by using**:

* own implementation 

We leave it to the user to use scikit-learn library - see https://scikit-learn.org/stable/modules/generated/sklearn.inspection.partial_dependence.html#sklearn.inspection.partial_dependence 

In [ ]:
""" This cell calculates the partial dependency plot for all model input variables
"""

#Let's take 10 random observations for which we want to plot the ICE
random_indices = np.random.randint(0,len(df),10)

for var in inputs:

    if var in continuous_inputs:

        var_range = np.linspace(df[var].min(),df[var].max(),100) #define it's range 

    elif var in categorical_inputs:

        var_range = np.unique(df[var].dropna())

    ICE_vals = np.zeros((len(df) , len(var_range))) #For each observation in the dataset we store the ICE. The PDP is obtained later on by averaging these ICE's.
    i = 0
    for val in var_range:

        df_ = df.copy()

        #Replace all observations with the same value for 'age'
        df_.loc[:,var] = val

        #Predict model outcome using the dataset in which all observations have the same age 'val' (leaving all other variables untouched).
        predictions = model_to_explain.predict_proba(df_[inputs])[:,1]

        #Let's save the predictions for age 'val' for every observation.
        ICE_vals[:,i] = predictions
        i = i+1

    """
    When we plot the prediction outcome for a single observation over the range of the feature, we obtain the so-called "Individual Conditional Expectation' (ICE) plot
    For a given value of the feature, taking the average of the observations, gives us the PDP. The PDP is thus the average of ICE's.

    Below, let's sample some observations to plot their ICE and overlay with the PDP

    """
   
    samples_to_plot = ICE_vals[random_indices,:]

    # #Let's plot the PDP figure to compare with the above figures.
    fig = make_subplots()

    if var in continuous_inputs:
        for k in range(len(random_indices)):
            fig.add_trace(go.Scatter(x = var_range, y = samples_to_plot[k,:], name = 'ICE observation index. ' + str(random_indices[k])))

    elif var in categorical_inputs:        
        for k in range(len(random_indices)):
            fig.add_trace(go.Bar(x = var_range, y = samples_to_plot[k,:], name = 'ICE observation index. ' + str(random_indices[k])))


    fig.add_trace(go.Scatter(x = var_range, y = np.mean(ICE_vals,axis=0),name = 'PDP',line = dict(color='black', width=4,dash='dashdot')))
    fig.update_xaxes(title_text = var)
    fig.update_yaxes(title_text = 'PREDICTED probability of ' + target)
    fig.update_layout(title_text = "ICE + Partial Dependency Plot of model variable " + var)
    fig.show()

The ICE shows that for some inviduals the predicted outcome as a function of age is (slightly) different as compared to others (more steep, or more flat). The average of alle the invididual curves in the dataset corresponds to the Partial Dependency Plot.

# DISCUSSION ON THE OUTCOME OF THE PDP ON THE TITANIC DATASET

We analyze the PDP for each feature to assess whether the model exhibits intuitive behaviour.

|Variable | Description of PDP | Intuitive? |
| :- | :- |:-|
|Pclass | For lower Pclass, the model predicts a lower probability of survival. The prob of survival for Pclass = 1 ('First class') is approx. double of Pclass = 3 ('Third class') | Yes. Third class is mostly lower deck for which chances on survival are lower. |
|Sex | Male only predicted 20% of survival as compared to 70% for women | Yes. "Women and children first" seems to hold, also by the model. Note also that the large difference in predicted survival probability is consistent with the feature importance as measured in the previous section.|
|Age | Older people are predicted lower probability of survival as compared to younger people. The ratio of prob. of predicted survival between youngest and eldest is approx. 2.7 (0.55 / 0.2) | Yes. |
|SibSp | When the number of Siblings and spouses travelling together with the passenger is between 0 and 2, the predicted probability of survival is approx. 35%. For higher numbers (>=3), this probability suddenly drops to approx. 20%. | Yes. We might assume some correlation effect with the Pclass variable (to be investigated) in which passengers that can afford better class tend to travel with fewer people, while poorer people tend to travel with larger groups|
|Parch | The number of parents and children travelling along with the passenger shows similar behaviour to the 'SibSp' variable. Although here the difference in predicted prob. of survival is smaller between 'up to 3' groups and larger (>=4) groups. |Yes. See SibSp variable |
|Fare | The higher the fare (price of ticket) the higher the predicted prob. of survival. | Yes. Probably also related to 'Pclass' 
|Embarked | Passengers embarked in Cherbourg (C) have highest predicted prob. of survival, followed by Queenstown (Q) and Southampton (S). However the difference can be deemed rather small (45% vs. 37%). | Yes, in that sense that we do not really expact a causal dependence of embarkment place on prob. of survival, thus meaning the PDP is rather 'flat' as a function of embarkment |


<a id="shapley"></a>
# [Shapley values](#toc)

see also https://christophm.github.io/interpretable-ml-book/shapley.html.

**General** <br>

Game-theoratical approach in which we want to explain what features are responsible for the predicted outcome for a given single observation of interest.<br>

Following reasoning is used:

* Given all our observations in the dataset, we have the average predicted outcome over all these observations.
* Now imagine the model variables (features) as players, all playing a joint game in which they want to realize a gain (i.e. all features contribute for a single observation to get a predicted outcome for this particular observation. The players are the features. The game is making a prediction). Predictions with a negative (positive) gain have a predicted outcome lower (higher) than the average of the dataset.
* Shapley values now correspond to explaining 'how much each feature contributed in making the individual prediction deviate from the dataset average'. Game-theoretically: how much did each player contribute to the gain? Some might have had a positive contribution while others contributed negatively. <br>

The latter implies the following sum rule (with SV the Shapley Value):

$\sum_{f \in \{features\}}\textrm{SV}_i(f) = \textrm{prediction}_i - \langle\textrm{prediction}\rangle_i$

**How to calculate the Shapley value of a single feature?**

Say we have the set of features (players) $f = \{f_1, f_2, \ldots , f_N \}$ <br>
While playing the game, some players might decide to form so-called coalitions by pairing up with each other without taking care of the others. We thus have a subset of the set of features, for example $f_{coalition} = \{ f_2, f_4 \} \subset f$. <br>
We now only focus on this particular coalition, which means we average out (marginalize) all other features (in the numerical calculation we replace them by random features of other observations).<br>
Now, say we want to calculate the Shapley value of feature $f_2$ (each feature has it's own Shapley value with the above sum rule). Then we need to do the following:
1. For the given coalition, calculate the marginal impact of the feature of interest by making the prediction with $f_2$ and by marginalizing it out of the coalition. The latter is again done in the numerical calculation by replacing $f_2$ with a value of another random observation in the dataset. This difference in prediction with and without the feature of interest corresponds to the *'marginal contribution of $f_2$ within the given coalition'*. Off course, in practice this result depends on what random features have been imputed here, so we need to an additional averaging her to get stable results.
2. Do step 1 for all possible coalitions (this is $2^N$ if we have N features!) and calculate the average marginal contribution over all coalitions. That's the Shapley value.

## Brute force implementation

We will implement - however completely not optimized in terms of efficiency, speed etc. - the above narrative. Next, we will use some open-source available library (https://github.com/slundberg/shap)
which has more powerful implementations and visualizations.

In [ ]:
#Function that helps us to retrieve all possible subsets of a given list.
def powerset(seq):
    """
    Returns all the subsets of this set. This is a generator.
    """
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item

In [ ]:
#All possible coalitions (subsets) of the given model inputs.

if len(inputs) <= 8:
    #If number of inputs is smaller than 9, we calculate all possible calculations and use them all
    coalitions = [x for x in powerset(inputs)]
else:
    #If number of inputs is too large it's impossible to loop over all coalitions. As a workaround: take 2^8 = 256 random coalitions
    
    coalitions = []
    for k in np.arange(2**8):
        mask = np.round(np.random.random((1,len(inputs)))).flatten().tolist()
        bool_list = list(map(bool,mask))
        coalition_to_add = [item for keep, item in zip(bool_list,inputs) if keep]
        coalitions.append(coalition_to_add)

    print("Too many coalitions possible. Random samples of all possible coalitions used. Reside to the built-in SHAP library as an alternative.")

In [ ]:
#Take a random observation
ooi = df.sample()[inputs] #single Observation Of Interest (ooi) that we want to explain.

print('Index observation: ' + str(ooi.index.values))

shapley_vals = np.zeros((1,len(inputs)))

M = 50 #Number of random observations to average over per coalition
i = 0

for var in inputs:
    print(var)
    shapley = 0

    for coal in coalitions:
       
        vars_not_in_coal = [item for item in inputs if item not in coal]
            
        #We need to make a prediction of the coalition by having the original feature value itself (i.e. we need to ensure not to replace it with a random value from the beginning). 
        if (var in vars_not_in_coal):
            vars_not_in_coal.remove(var) 
        
        z = df.sample(M)[inputs] #another random sample to mimick the 'coalition' in the next rule (i.e. to marginalize all non-coalition members)
        x = pd.concat([ooi]*M, ignore_index=True)

        if len(vars_not_in_coal) > 0:
            x[vars_not_in_coal] = z[vars_not_in_coal].values
            
        #MAKE A PREDICTION WITH THE FEATURE IN THE COALITION
        pred_var_in_coalition = model_to_explain.predict_proba(x[inputs])[:,1]

        #MAKE A PREDICTION WITHOUT THE FEATURE IN THE COALITION (i.e. replace the feature of interest with the value of the random observation z)
        x[var] = z[var].values
        pred_var_not_in_coalition = model_to_explain.predict_proba(x[inputs])[:,1]

        #Add the average marginal contribution of the given coalition to the counter  
        shapley =  shapley + np.mean(pred_var_in_coalition - pred_var_not_in_coalition) 

    shapley_vals[0,i] = shapley/len(coalitions)
    i = i+1
    
shapley_vals = pd.DataFrame(data = shapley_vals , columns = inputs) 

In [ ]:
"""SHAPLEY VALUES SHOULD OBEY A SUM RULE: sum of Shapley values = prediction of obs. i - average prediction over dataset"""
check_shapley = pd.DataFrame(columns = [ "Sum Shapley values (A)" ,  "Avg. prediction over dataset (B)" , "A+B" ,"Indiv. prediction"] )
check_shapley["Indiv. prediction"] = model_to_explain.predict_proba(ooi[inputs])[:,1]
check_shapley["Sum Shapley values (A)"] = np.sum(shapley_vals, axis = 1)
check_shapley["Avg. prediction over dataset (B)"] = np.mean(model_to_explain.predict_proba(df[inputs])[:,1])
check_shapley["A+B"] = check_shapley["Avg. prediction over dataset (B)"] + check_shapley["Sum Shapley values (A)"] 
check_shapley["Rel. Difference"] = (check_shapley["A+B"] - check_shapley["Indiv. prediction"])/check_shapley["Indiv. prediction"]
check_shapley["Converged?"] = np.where((np.abs(check_shapley["Rel. Difference"]) < 0.05), True, False)

print("Check Shapley value sum rule (within numerical precision due to finite random sampling - at least the difference should be smaller than the sum of the Shapley values itself otherwise we need additional averaging or there's an error in the implementation)")
display(check_shapley)

In [ ]:
print("The Shapley values indicate which model variable contributed in which direction to make the individual prediction deviate from the dataset average (i.e. the sum rule)")
display(ooi)

In [ ]:
fig = make_subplots()
fig.add_trace(go.Bar(x = shapley_vals.columns.values , y = shapley_vals.values.flatten()))
fig.update_yaxes(title_text = 'Shapley value')

## Using 'SHAP' library

In [ ]:
x_Train = model_to_explain.named_steps[preprocessor].transform(df[inputs])
explainer_ = shap.Explainer(model_to_explain.named_steps[classifier],x_Train,link = shap.links.logit)
shap_vals = explainer_(x_Train)

In [ ]:
#Because Shap uses the logit (and not the probability) write a function that transformed the x-axis for the waterfall plot to probabilities
def xgb_shap_transform_scale(original_shap_values, Y_pred, which):
    from scipy.special import expit
    
    #Compute the transformed base value, which consists in applying the logit function to the base value
    from scipy.special import expit #Importing the logit function for the base value transformation
    untransformed_base_value = original_shap_values.base_values[-1]
   
    #Computing the original_explanation_distance to construct the distance_coefficient later on
    original_explanation_distance = np.sum(original_shap_values.values, axis=1)[which]
    
    base_value = expit(untransformed_base_value ) # = 1 / (1+ np.exp(-untransformed_base_value))

    #Computing the distance between the model_prediction and the transformed base_value
    distance_to_explain = Y_pred[which] - base_value

    #The distance_coefficient is the ratio between both distances which will be used later on
    distance_coefficient = original_explanation_distance / distance_to_explain

    #Transforming the original shapley values to the new scale
    shap_values_transformed = original_shap_values / distance_coefficient

    #Finally resetting the base_value as it does not need to be transformed
    shap_values_transformed.base_values = base_value
    shap_values_transformed.data = original_shap_values.data
    
    #Now returning the transformed array
    return shap_values_transformed  

In [ ]:
shap.plots.beeswarm(shap_vals)

In [ ]:
print('run succesful')